<a href="https://colab.research.google.com/github/jlonge4/gen_ai_utils/blob/main/LLM_loaders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install langchain farm-haystack faiss-gpu boto3 unstructured python-pptx

In [ ]:
import boto3
bedrock = boto3.client('bedrock-runtime',
                       region_name='us-east-1')
from langchain.llms import Bedrock
model_kwargs =  {
    "max_tokens_to_sample": 8191,
    "temperature": 0.5,
    "top_k": 500,
    "top_p": 1,
}

llm = Bedrock(client=bedrock, model_id='anthropic.claude-v2', model_kwargs=model_kwargs, verbose=False)

LANNGCHAIN CSV LOADER

In [ ]:
from langchain.document_loaders.csv_loader import CSVLoader
loader = CSVLoader(file_path="/content/sample_data/bus_earnings_6_2_23.csv")

data = loader.load()

In [ ]:
data[:3]

[Document(page_content=': 0\nWEEK: 2023-02-18T20%3A40%3A37.259274Z\nEARNINGS: 1976.85', metadata={'source': '/content/sample_data/bus_earnings_6_2_23.csv', 'row': 0}),
 Document(page_content=': 1\nWEEK: 2023-03-27T12%3A00%3A34.272607Z\nEARNINGS: 278.1', metadata={'source': '/content/sample_data/bus_earnings_6_2_23.csv', 'row': 1}),
 Document(page_content=': 2\nWEEK: 2023-04-03T12%3A00%3A34.290030Z\nEARNINGS: 509.85', metadata={'source': '/content/sample_data/bus_earnings_6_2_23.csv', 'row': 2})]

In [ ]:
resp = llm(f'What were the earnings for May 1st using this data <data>{data}</data>')

In [ ]:
resp

" Based on the provided data, the earnings for May 1st, 2023 are 345.05. This can be found in the document with metadata {'row': 5}:\n\nDocument(page_content=': 5\nWEEK: 2023-05-01T12%3A00%3A34.370572Z  \nEARNINGS: 345.05', metadata={'source': '/content/sample_data/bus_earnings_6_2_23.csv', 'row': 5})\n\nThe key information is the date 2023-05-01 in the WEEK field, which indicates this document contains data for May 1st. The corresponding EARNINGS field shows the earnings for that date were 345.05."

LANGCHAIN POWERPOINT LOADER

In [ ]:
from langchain.document_loaders import UnstructuredPowerPointLoader
loader = UnstructuredPowerPointLoader("/content/sample_data/D209 Data Mining 1 Task 1 Cohort.pptx")
# This comes out as a massive chunk of text. So it will need to be preprocessed accordingly.
data = loader.load()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
llm('Using the data {data} \n\n answer the question \n\n how do i get access to the course folder?')

' Unfortunately the data provided does not contain enough information to directly answer how to access a specific course folder. However, here are a few general tips on accessing course folders:\n\n- Course folders are usually located in the learning management system (LMS) used by the school, such as Blackboard, Canvas, Moodle etc. You would need an account and login credentials for the LMS to access course folders.\n\n- The instructor may share a direct link to the course folder. This allows access without logging into the full LMS.\n\n- Course folders may be hosted on a separate file storage system and the instructor would need to provide the URL or login details.\n\n- Some common locations for course folders include inside the LMS course page, under "course documents" or "course files", in a module named "course materials", etc.\n\n- You can try searching the LMS course pages or asking the instructor directly where the folder is located and how to access it. \n\n- Bookmarking or sa

HAYSTACK PDF LOADER WITH OCR FOR IMAGE DATA



In [ ]:
pip install farm-haystack[pdf] pytesseract

In [ ]:
from haystack.pipelines import Pipeline
from haystack.nodes import PDFToTextConverter, PreProcessor


pdf_converter = PDFToTextConverter(ocr="full")
preprocessor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=False,
    split_by="word",
    split_length=500,
    split_respect_sentence_boundary=True,
)

# This is an indexing pipeline
p = Pipeline()

p.add_node(component=pdf_converter, name="PdfConverter", inputs=["FileTypeClassifier.output_2"])

p.add_node(
    component=preprocessor,
    name="Preprocessor",
    inputs=["PdfConverter"],
)

docs = p.run(file_paths=["/content/sample_data/Dr. Middleton Getting Started with D208(Part I)COIT.pdf"])

LANGCHAIN PDF WITH IMAGES

In [ ]:
pip install rapidocr-onnxruntime pypdf


In [ ]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("/content/sample_data/Dr. Middleton Getting Started with D208(Part I)COIT.pdf", extract_images=True)
pages = loader.load()
pages[4].page_content

'Dr. Eric \nStraw\nDr. Keiona \nMiddleton\nIf your assigned Course Instructor (CI) is not available, feel free to schedule time with any (CI) that supports this course.  \nVisit “Announcements” within the course \nto book with any CI.Meet Your Course Instructors\nDr. William \nSewell\nDr. Daniel \nSmith\nWGUUTCOLLEGEOFINFORMATIONTECHNOLOGY'

In [ ]:
pages[20].page_content

'D208 | Predictive Modeling\nYOU ARE HERE!•Predictive Modeling (using Regression) is a statistical method that helps us \nto analyze and understand the relationship between two or more variables \nof interest.\n•The process helps to understand which factors are important, which factors \ncan be ignored, and how they are influencing each other .\n•Therefore, your goal in this course is to:\n•Multiple Linear Regression\n•Logistic Regression\n•In this course, you will either use Python or R to conduct regression analysis. \n•Your competence will be demonstrated by the successful completion of the \nperformance assessment.Data Analytics \nLife CycleWGUUTCOLLEGEOFINFORMATIONTECHNOLOGYReporting\nBusiness\nAnd\nUnderstanding\nVisualization\nData Mining\nData\nMachine\nAcquisition\nLearning\nPredictive\nData\nModeling\nCleaning\nData\nExploration'